In [172]:
import pickle as pkl
import pandas as pd
import numpy as np
import datetime as dt
import re
import sys

In [188]:
donors = pd.read_pickle('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/df_donor_clean-Copy1.pkl')
c_approval = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_c_approval_Final.csv')
p_approval = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_p_approval_Final.csv')
generic_approval = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_Generic_Final.csv')
direction_approval = pd.read_csv('/Users/Brandon/indEng_data_x/tenguins/data/cleaned/RCP_Direction_Final.csv')

#Copies
minimum_num_donors = donors
congress = c_approval
president = p_approval
generic = generic_approval
direction = direction_approval

In [195]:
####################
# helper functions #
####################


flatten = lambda lst: [item for sublist in lst for item in sublist]  


def containsAny(str, set):
    return 1 in [element in str for element in set]


def quantizeSpread(lst):
    disapprovalSet = {'disapprove', 'Disapprove', 'Wrong Direction', 'Wrong Track'}
    tieSet = {'Tie', 'tie'}
    for i in range(len(lst)):
        if containsAny(lst[i], disapprovalSet):
            lst[i] = re.sub(r'[^0-9]+', "-", lst[i])
        elif containsAny(lst[i], tieSet):
            lst[i] = re.sub(r'[^0-9]+', "0", lst[i])
        else:
            lst[i] = re.sub(r'[^0-9]', "", lst[i])     
    if flatten([re.findall(r'[^0-9\-]', i) for i in lst]):
        raise Exception(lst)
    return lst

def approvalTableCleanup(table, approvalType):
    splitColumn = [i.split(',') for i in table['Results']]
    approvalCol = [splitColumn[i][0] for i in range(len(splitColumn))]
    disapprovalCol = [splitColumn[i][1] for i in range(len(splitColumn))]
#     table['approval rating'] = [re.sub(r'[^0-9]', "", i) for i in approvalCol]
    approvalCol = [re.sub(r'[^0-9]', "", i) for i in approvalCol]
    table['approval rating'] = [int(i) for i in approvalCol]
#     table['disapproval rating'] = [re.sub(r'[^0-9]', "", i) for i in disapprovalCol]
    disapprovalCol = [re.sub(r'[^0-9]', "", i) for i in disapprovalCol]
    table['disapproval rating'] = [int(i) for i in disapprovalCol]
    table = table.drop('Results', axis = 1)
    table = table.drop('Poll', axis = 1)
    table = table.drop('Year', axis = 1)
    stringSpreads = table['Spread'].tolist()
    spreadCol = quantizeSpread(stringSpreads)
    table['spread'] = [int(i) for i in spreadCol]
    table = table.drop('Spread', axis = 1)
    table.columns = map(str.lower, table.columns)
    table = table.rename(index = str, columns = {"race/topic (click to sort)": "approval type"})
    table['approval type'] = table['approval type'].str.lower()
    table = table.set_index('date')
    table = table.groupby('date').mean()
    table['approval type'] = [approvalType for i in range(len(table))]
    return table

def stack_cpgd_data(frames):
    agg_table = pd.concat(frames)
    return agg_table

donorLevels = dict([('d1', 200), ('d2', 499.99), ('d3', 999.99), ('d4', 1999.99), ('d5', 2000)])
adHeur = lambda col, level: np.divide(col, donorLevels[level])

def donor_level_features(table):
    
    """Adds features to the FEC table that include the minimum number of possible donors
    per donor level"""
    
    adHeur = lambda col, level: np.divide(col, donorLevels[level])
    donorLevels = dict([('d1', 200), ('d2', 499.99), ('d3', 999.99), ('d4', 1999.99), ('d5', 2000)])
    
    table['estimated_num_d1_donors'] = adHeur(table['Donation Level 1'], 'd1')
    table['estimated_num_d2_donors'] = adHeur(table['Donation Level 2'], 'd2')
    table['estimated_num_d3_donors'] = adHeur(table['Donation Level 3'], 'd3')
    table['estimated_num_d4_donors'] = adHeur(table['Donation Level 4'], 'd4')
    table['estimated_num_d5_donors'] = adHeur(table['Donation Level 5'], 'd5')
    return table


In [191]:
c_final = approvalTableCleanup(congress, 'Congressional Job Approval')
p_final = approvalTableCleanup(president, 'President Obama Job Approval')
g_final = approvalTableCleanup(generic, 'Generic Congressional Vote')
d_final = approvalTableCleanup(direction, 'Direction of Country')

In [192]:
cpgd_final_frames = [c_final, p_final, g_final, d_final]
cpgd_final = stack_cpgd_data(cpgd_final_frames)

In [194]:
cpgd_final.to_pickle('cpgd_actual_final.pkl')

In [198]:
len(cpgd_final)

3341

In [150]:
# direction of country
# congressional approval
# presidential approval
# generic congressional vote


#Copies
# minimum_num_donors = donors
# congress = c_approval
# president = p_approval
# generic = generic_approval
# direction = direction_approval

In [ ]:
cpgd_copy = cpgd_copy.set_index('date')
cpgd_copy = cpgd_copy.groupby('date').mean()

In [125]:
c_approval.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,Congressional Job Approval,Quinnipiac,"Approve 20, Disapprove 71",Disapprove +51,2010-04-21,2010
1,Congressional Job Approval,FOX News,"Approve 22, Disapprove 68",Disapprove +46,2010-04-22,2010
2,Congressional Job Approval,NBC News/Wall St. Jrnl,"Approve 21, Disapprove 72",Disapprove +51,2010-05-12,2010
3,Congressional Job Approval,Gallup,"Approve 21, Disapprove 75",Disapprove +54,2010-05-14,2010
4,Congressional Job Approval,Associated Press/GfK,"Approve 28, Disapprove 71",Disapprove +43,2010-05-15,2010


In [126]:
p_approval.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,President Obama Job Approval,Quinnipiac,"Approve 44, Disapprove 46",Disapprove +2,2010-04-21,2010
1,President Obama Job Approval,FOX News,"Approve 46, Disapprove 46",Tie,2010-04-22,2010
2,President Obama Job Approval,Gallup,"Approve 50, Disapprove 43",Approve +7,2010-04-25,2010
3,President Obama Job Approval,Rasmussen Reports,"Approve 45, Disapprove 54",Disapprove +9,2010-04-25,2010
4,President Obama Job Approval,Democracy Corps (D),"Approve 48, Disapprove 46",Approve +2,2010-04-27,2010


In [127]:
generic.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,Generic Congressional Vote,PPP (D),"Republicans 47, Democrats 42",Republicans +5,2010-04-16,2010
1,Generic Congressional Vote,Gallup,"Republicans 46, Democrats 43",Republicans +3,2010-04-19,2010
2,Generic Congressional Vote,Rasmussen Reports,"Republicans 46, Democrats 36",Republicans +10,2010-04-19,2010
3,Generic Congressional Vote,FOX News,"Republicans 42, Democrats 38",Republicans +4,2010-04-22,2010
4,Generic Congressional Vote,Rasmussen Reports,"Republicans 44, Democrats 38",Republicans +6,2010-04-26,2010


In [128]:
direction.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,Direction of Country,R2000/Daily Kos (D),"Right Direction 38, Wrong Track 58",Wrong Track +20,2010-04-16,2010
1,Direction of Country,R2000/Daily Kos (D),"Right Direction 39, Wrong Track 57",Wrong Track +18,2010-04-23,2010
2,Direction of Country,Rasmussen Reports,"Right Direction 33, Wrong Track 62",Wrong Track +29,2010-04-26,2010
3,Direction of Country,Democracy Corps (D),"Right Direction 34, Wrong Track 57",Wrong Track +23,2010-04-27,2010
4,Direction of Country,Rasmussen Reports,"Right Direction 33, Wrong Track 61",Wrong Track +28,2010-04-27,2010


In [130]:
# people = lambda people: 'franken' in people
# mask = minimum_num_donors.race_candidates.apply(people)
# temp = minimum_num_donors[mask]
# temp

# # def test():
# #     if name != winner:

# temp.columns

In [133]:
congress.head()

,Race/Topic (Click to Sort),Poll,Results,Spread,Date,Year
0,Congressional Job Approval,Quinnipiac,"Approve 20, Disapprove 71",Disapprove +51,2010-04-21,2010
1,Congressional Job Approval,FOX News,"Approve 22, Disapprove 68",Disapprove +46,2010-04-22,2010
2,Congressional Job Approval,NBC News/Wall St. Jrnl,"Approve 21, Disapprove 72",Disapprove +51,2010-05-12,2010
3,Congressional Job Approval,Gallup,"Approve 21, Disapprove 75",Disapprove +54,2010-05-14,2010
4,Congressional Job Approval,Associated Press/GfK,"Approve 28, Disapprove 71",Disapprove +43,2010-05-15,2010
